In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
import math,copy,time

In [3]:
from torch.autograd import Variable

In [4]:
import matplotlib.pyplot as plt
import seaborn

In [6]:
seaborn.set_context(context = 'talk')
%matplotlib inline

In [7]:
class EncoderDecoder(nn.Module):
    def __init__(self,encoder,decoder,src_embed,tgt_embed,generator):
        super(EncoderDecoder,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
      
    def forward(self,src,tgt,src_mask,tgt_mask):
        return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)
    
    def encode(self,src,src_mask):
        return self.encoder(self.src_embed(src),src_mask)
    
    def decode(self,memory,src_mask,tgt,tgt_mask):
        return self.decoder(self.tgt_embed(tgt),memory,src_mask,tgt_mask)
        

In [8]:
class Generator(nn.Module):
    def __init__(self,d_model,vocab):
        super(Generator,self).__init__()
        self.proj = nn.Linear(d_model,vocab)
       
    def forward(self,x):
        return F.log_softmax(self.proj(x),dim=-1)

In [9]:
def clones(module,N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [11]:
class Encoder(nn.Module):
    def __init__(self,layer,N):
        super(Encoder,self).__init__()
        self.layers = clones(layers,N)
        self.norm = LayerNorm(layer.size)
       
    def forward(self,x,mask):
        for layer in self.layers:
            x = layer(x,mask)
         
        return self.norm(x)

In [12]:
class LayerNorm(nn.Module):
    def __init__(self,features,eps = 1e-6):
        super(LayerNorm,self).__init__()
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
       
    def forward(self,x):
        mean = x.mean(-1,keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a2 * (x-mean)/(std + self.eps) + self.b2
    
    

In [14]:
class SublayerConnection(nn.Module):
    def __init__(self,size,dropout):
        super(SublayerConnection,self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x,sublayer):
        return x + self.dropout(sublayer(self.norm(x)))
           
        

In [15]:
class EncoderLayer(nn.Module):
    def __init__(self,size,self_attn,feed_forward,dropout):
        super(EncoderLayer,self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size,droput),2)
        self.size = size
       
    def forward(self,x,mask):
        x = self.sublayer[0](x, lambda x:self.self_attn(x,x,x,mask))
        return self.sublayer[1](x,feed_forward)
       
    

In [16]:
class Decoder(nn.Module):
    def __init__(self,layer,N):
        super(Decoder,self).__init__()
        self.layers = clones(layer,N)
        self.norm = LayerNorm(layers.size)
       
    def forward(self,x,memory,src_mask,tgt_mask):
        for layer in self.layers:
            x = layer(x,memory,src_mask,tgt_mask)
           
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,size,self_attn,src_attn,feed_forward,dropout):
        super(DecoderLayer,self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.dropout = dropout
        self.sublayer = clones(SublayerConnection(size,self.dropout),3)
       
    def forward(self,x,memory,src_mask,tgt_mask):
        m = memory
        x = self.sublayer[0](x,lambda x:self.src_attn(x,x,x,tgt_mask))
        x = self.sublayer[1](x,lambda x:self.src_attn(x,m,m,src_mask))
        
        
       
    

        